In [4]:
import json
from elasticsearch import Elasticsearch
from credentials import auth
from elasticsearch.helpers import scan
from elasticsearch_dsl import Search, Q

In [5]:
u,p=auth()
hosts ="https://ctesm1.discoverdollar.com:41532/"
es = Elasticsearch(hosts,  timeout=60000, use_ssl=True, verify_certs=False, 
ssl_show_warn=False, raise_on_error=False,http_auth=(u,p))

In [6]:
es.ping()

True

In [7]:
index="cantire-2020-06"
body={"size":1000,
      "query":{
    "match_phrase":{
    "fileType": "Message/Email File"
    }}
}
res=es.search(index=index,body=body)

D:\Users\sourav.kumar\AppData\Local\Temp\2/ipykernel_10824/2159714140.py:8: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res=es.search(index=index,body=body)


In [8]:
print(len(res['hits']['hits']))

1000


In [130]:
import re

import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_trf")


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|_\-\–<>;:]')
emails=re.compile('[\w.+-]+@[\w-]+\.[\w.-]+')
anchor=re.compile("<[^ ]*>")
cid=re.compile("cid[^ ]*")
sqbrac=re.compile('\[[^ ]*\]')
has=re.compile("#[0-9a-bA-B]*")
def prepare_text(text):
    #text = text.lower()
    text=anchor.sub(" ",text)
    text=sqbrac.sub(" ",text)
    text=has.sub(" ",text)
    text=cid.sub(" ",text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text=" ".join(text.split())
    return text
def get_35(t):
    k=t.split()
    if len(k)<=35:
        return t
    else:
        return " ".join(k[len(k)-35:])

In [131]:
print(res['hits']['hits'][0]['_source']['content'])

Hi Harpreet,



1.     After rechecked the consumption, please find the revised costing sheet as attached for reference.



2.     We quoted two prices based on “Brand Snap button” and “Non-brand Snap button” for your comparison, please reconfirm which version to be proceeded for bulk.



3.     For mold charge, it is die mold charge for the snap button.



4.     According to CTC’s policy that garment test is required for each shipment, so we add the garment test fee for the gaiter based on 1,400 pcs. Please let us know if garment test will not be required for this gaiter shipment, then we will remove the garment test from costing.



5.     Please confirm that all the details (incl. spec/ color combination/ workmanship) will be followed exactly as the gaiter sample which we sent on 5/13 for bulk production, and please provide us the tech pack with logo placement of the heat transfer logo, then we can proceed PP sample for approval directly.



6.     Based on order confirmed by 6/10,

In [132]:
content=res['hits']['hits'][1]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
print(l)

["The two SKUs in green have been SD'd. The Bloem SKU is a component SKU that they confirmed can't be ordered on it's own. Thanks, Valentina Juzukonis Replenishment Analyst Canadian Tire Retail, Ltd. 416.480.9780 ext. 23652", "Juzukonis Subject RE SD Request Don't SD anything Thermacell yet. No reason to SD the Bloem sku can they not keep up with supply? 2 green skus you can SD. Chris Stewart Category Business Manager Gardening Canadian Tire Corporation 416.544.6057", '1590910 REFILL 120 HR 35.95 G49N THERMACELL REPELLENT ACT N 05 05 6 7,523 7,517 6,936 581 Thanks, Valentina Juzukonis Replenishment Analyst, Seasonal & Gardening Canadian Tire Retail 2180 Yonge St. Toronto, ON, M4P 2V8 T 416.480.9780 ext. 23652 Valentina.Juzukonis@cantire.com']


In [143]:
content=res['hits']['hits'][1]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    print(displacy.render(doc,style="ent"))
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

None
Valentina Juzukonis Replenishment Analyst Canadian Tire Retail, Ltd. 416.480.9780 ext. 23652
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
Chris Stewart Category Business Manager Gardening Canadian Tire Corporation 416.544.6057
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
Valentina Juzukonis Replenishment Analyst, Seasonal & Gardening Canadian Tire Retail 2180 Yonge St. Toronto, ON, M4P 2V8 T 416.480.9780 ext. 23652 Valentina.Juzukonis@cantire.com
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [142]:
content=res['hits']['hits'][1]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    #print(displacy.render(doc,style="ent"))
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    if person:
        cardinals=re.findall('\d{3}[\. ]?\d{3}[\. ]?\d{4}|\d{2} \d{8}',s)
        if cardinals:
            doc=nlp(s[person[-1][0]:])
            print(displacy.render(doc,style="ent"))
    print()
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [134]:
content=res['hits']['hits'][50]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    print(displacy.render(doc,style="ent"))
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

None
Alexi Alexi Plut Category Business Manager White Goods, Cutlery & ASOTV Canadian Tire Corporation T 416.544.7783 C 416.529.9883
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
Alexi Plut Category Business Manager White Goods, Cutlery & ASOTV Canadian Tire Corporation T 416.544.7783 C 416.529.9883
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
s disappointing. Jeff Race Dealer Canadian Tire 124 465 Norwich Avenue Woodstock, Ont N4S 9A2 519 539 1201 x231 jeff.119@hotmail.com
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
Alexi Plut Category Business Manager White Goods, Cutlery & ASOTV Canadian Tire Corporation T 416.544.7783 C 416.529.9883
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
Alexi Alexi Plut Category Business Manager White Goods, Cutlery & ASOTV Canadian Tire Corporation T 416.544.7783 C 416.529.9883
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
Alexi Plut Subject Fwd 399 6488 MC KEG forecast too low. FYI can you look at this Get Outlook for iOS
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
Ward, KL Sent from my BlackBerry the most secure mobile device via the Bell Network
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
431 3576 Cell 778 836 8349 wardsaitogs@gmail.com Canadian Tire and Olympics
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
Race Dealer Canadian Tire 124 465 Norwich Avenue Woodstock, Ont N4S 9A2 519 539 1201 x231 jeff.119@hotmail.com
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None
431 3576 Cell 778 836 8349 wardsaitogs@gmail.com Canadian Tire and Olympics
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [148]:
content=res['hits']['hits'][50]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    person=[]
    doc=nlp(s)
    #print(displacy.render(doc,style="ent"))
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    if person:
        cardinals=re.findall('\d{3}[\. ]?\d{3}[\. ]?\d{4}|\d{2} \d{8}',s[person[-1][0]:])
        if cardinals:
            doc=nlp(s[person[-1][0]:])
            print(displacy.render(doc,style="ent"))
    print()
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [149]:
content=res['hits']['hits'][100]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    person=[]
    doc=nlp(s)
    #print(displacy.render(doc,style="ent"))
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    if person:
        cardinals=re.findall('\d{3}[\. ]?\d{3}[\. ]?\d{4}|\d{2} \d{8}',s[person[-1][0]:])
        if cardinals:
            doc=nlp(s[person[-1][0]:])
            print(displacy.render(doc,style="ent"))
    print()
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


...

None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [150]:
content=res['hits']['hits'][150]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    person=[]
    doc=nlp(s)
    #print(displacy.render(doc,style="ent"))
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    if person:
        cardinals=re.findall('\d{3}[\. ]?\d{3}[\. ]?\d{4}|\d{2} \d{8}',s[person[-1][0]:])
        if cardinals:
            doc=nlp(s[person[-1][0]:])
            print(displacy.render(doc,style="ent"))
    print()
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [151]:
content=res['hits']['hits'][200]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    person=[]
    doc=nlp(s)
    #print(displacy.render(doc,style="ent"))
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    if person:
        cardinals=re.findall('\d{3}[\. ]?\d{3}[\. ]?\d{4}|\d{2} \d{8}',s[person[-1][0]:])
        if cardinals:
            doc=nlp(s[person[-1][0]:])
            print(displacy.render(doc,style="ent"))
    print()
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


...

None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


None

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [138]:
content=res['hits']['hits'][250]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

Zack Zeraldo Category Business Manager Brakes Canadian Tire Corporation C 416.795.8984
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Zachary Zeraldo Category Business Manager Brakes Canadian Tire Retail 2180 Yonge Street, Toronto, ON, M4P 2V8 T 416.480.3723 C 416.795.8984 Zachary.Zeraldo@cantire.com
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [139]:
content=res['hits']['hits'][300]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

Evan, We have a call scheduled for the 5th…. But can we jump on a catch up call around the men’s business? Wondering if you are available shortly to connect?
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Evan Cole Territory Business Director, Canada Burton Americas C 647 500 3425 Burton.com Facebook Twitter Instagram
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Jeffrey Perrotte l Associate Vice President Purchasing B 403.592.8290 ext 80345 l C 403.390.0310 Email jeffrey.perrotte@fglsports.com
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Evan Cole Territory Business Director, Canada Burton Americas C 647 500 3425 Burton.com Facebook Twitter Instagram
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [140]:
content=res['hits']['hits'][350]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

IndexError: list index out of range

In [ ]:
content=res['hits']['hits'][400]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][450]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][500]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][550]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][600]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][650]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][700]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][750]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][800]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][850]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][900]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[i.strip() for i in l]
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [ ]:
content=res['hits']['hits'][950]['_source']['content']
content.encode("ascii", "replace")
text=" ".join(content.split())
l=re.split("From:",text)
l=[prepare_text(i) for i in l]
l=[get_35(i) for i in l]
person=[]
for s in l:
    doc=nlp(s)
    
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person.append((ent.start_char, ent.end_char))
    print(s[person[-1][0]:])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

In [159]:
import json
def get_name(l):
    if len(l)>1:
        l=l[1:]
    names=[]
    for i in l:
        r=re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+',i if len(i)<100 else i[:100])
        if r:
            name=i[:r.start()]
            REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|\-\–<>;:]')
            name = REPLACE_BY_SPACE_RE.sub(' ', name)
            names.append(name)
            
        else:
            names.append("no-info")
            
for i in [0,1,50,100]:
        d=dict()
        content=res['hits']['hits'][i]['_source']['content']
        content.encode("ascii", "replace")
        text=" ".join(content.split())
        l=re.split("From:",text)
        l=[i.strip() for i in l]
        names=get_names(l)
        l=[prepare_text(i) for i in l]
        l=[get_35(i) for i in l]
    
        for i in len(l):
            s=l[i]
            d=dict()
            person=[]
            org=[]
            doc=nlp(s)
            num=[]
            for ent in doc.ents:
                if ent.label_=="PERSON":
                    person.append((ent.start_char, ent.end_char))
                elif ent.label_=="ORG":
                    org.append((ent.start_char, ent.end_char))
               
                
            if person:
                for j in range(len(org)):
                    if org[j][0]>person[-1][1]:
                        org=org[j:]
                        break
                else:
                    org=[]
               
                cardinals=re.findall('\d{3}[\. ]?\d{3}[\. ]?\d{4}|\d{2} \d{8}',s[person[-1][0]:])
                if cardinals:
                    ind=s.index(cardinals[0])
                    d['Name']=s[person[-1][0]:person[-1][1]+1]
                    if org:
                        d['Designation']=s[person[-1][1]+1:org[0][0]]
                        d['orgs']=s[org[0][0]:ind]
                    else:
                        d['org']=s[person[-1][0]:ind]
                    d['Contacts']=cardinals
            
            if d :           
                print(d)
            else:
                print(names[i])

{'Name': 'Valentina Juzukonis ', 'Designation': 'Replenishment Analyst ', 'orgs': 'Canadian Tire Retail, Ltd. ', 'Contacts': ['416.480.9780']}
{'Name': 'Chris Stewart ', 'Designation': 'Category Business Manager ', 'orgs': 'Gardening Canadian Tire Corporation ', 'Contacts': ['416.544.6057']}
{'Name': 'Valentina Juzukonis ', 'Designation': 'Replenishment Analyst, ', 'orgs': 'Seasonal & Gardening Canadian Tire Retail 2180 Yonge St. Toronto, ON, M4P 2V8 T ', 'Contacts': ['416.480.9780']}
{'Name': 'Alexi Alexi Plut ', 'Designation': 'Category Business Manager ', 'orgs': 'White Goods, Cutlery & ASOTV Canadian Tire Corporation T ', 'Contacts': ['416.544.7783', '416.529.9883']}
{'Name': 'Alexi Plut ', 'Designation': 'Category Business Manager ', 'orgs': 'White Goods, Cutlery & ASOTV Canadian Tire Corporation T ', 'Contacts': ['416.544.7783', '416.529.9883']}
{'Name': 'Alexi Plut ', 'Designation': 'Category Business Manager ', 'orgs': 'White Goods, Cutlery & ASOTV Canadian Tire Corporation T '

In [163]:
import json
for i in [0]:
        d=dict()
        content=res['hits']['hits'][0]['_source']['content']
        content.encode("ascii", "replace")
        text=" ".join(content.split())
        l=re.split("From:",text)
        l=[prepare_text(i) for i in l]
        l=[get_35(i) for i in l]
    
        for s in l:
            d=dict()
            person=[]
            org=[]
            doc=nlp(s)
            num=[]
            print(displacy.render(doc,style="ent"))
            for ent in doc.ents:
                if ent.label_=="PERSON":
                    person.append((ent.start_char, ent.end_char))
                elif ent.label_=="ORG":
                    org.append((ent.start_char, ent.end_char))
               
                
            if person:
                for j in range(len(org)):
                    if org[j][0]>person[-1][1]:
                        org=org[j:]
                        break
                else:
                    org=[]
               
                cardinals=re.findall('\d{3}[\. ]?\d{3}[\. ]?\d{4}|\d{2} \d{8}',s[person[-1][0]:])
                if cardinals:
                    ind=s.index(cardinals[0])
                    d['Name']=s[person[-1][0]:person[-1][1]+1]
                    if org:
                        d['Designation']=s[person[-1][1]+1:org[0][0]]
                        d['orgs']=s[org[0][0]:ind]
                    else:
                        d['org']=s[person[-1][0]:ind]
                    d['Contacts']=cardinals
            
                     
            print(d)

None
{}


None
{}


None
{}


None
{}


None
{}


...

None
{}


None
{}


None
{}


None
{}


None
{}
